In [ ]:
from teehr import Evaluation
from pathlib import Path
import shutil
import xarray as xr
import geopandas as gpd
from teehr.models.tables import (
    Attribute,
    Configuration,
    Variable
)

In [ ]:
# Set a path to the directory where the evaluation will be created
TEST_STUDY_DIR = Path(Path().home(), "temp", "test_study")
shutil.rmtree(TEST_STUDY_DIR, ignore_errors=True)
TEST_STUDY_DIR.mkdir(parents=True, exist_ok=True)

# Set a path to the directory where the test data is stored
# TEST_DATA_DIR = Path("/home/matt/repos/teehr/tests/data/v0_3_test_study")
TEST_DATA_DIR = Path("/home/sam/temp/test_study")
GEOJSON_GAGES_FILEPATH = Path(TEST_DATA_DIR, "geo", "gages.geojson")
PRIMARY_TIMESERIES_FILEPATH = Path(
    TEST_DATA_DIR, "timeseries", "test_short_obs.parquet"
)
CROSSWALK_FILEPATH = Path(TEST_DATA_DIR, "geo", "crosswalk.csv")
SECONDARY_TIMESERIES_FILEPATH = Path(
    TEST_DATA_DIR, "timeseries", "test_short_fcast.parquet"
)
GEO_FILEPATH = Path(TEST_DATA_DIR, "geo")

In [ ]:
# Create an Evaluation object
eval = Evaluation(dir_path=TEST_STUDY_DIR)

# Enable logging
eval.enable_logging()

# Clone the template
eval.clone_template()

In [ ]:
# Load the location data (observations)
eval.load.import_locations(in_path=GEOJSON_GAGES_FILEPATH)

In [ ]:
# Load the timeseries data and map over the fields and set constants
eval.load.import_primary_timeseries(
    in_path=PRIMARY_TIMESERIES_FILEPATH,
    field_mapping={
        "reference_time": "reference_time",
        "value_time": "value_time",
        "configuration": "configuration_name",
        "measurement_unit": "unit_name",
        "variable_name": "variable_name",
        "value": "value",
        "location_id": "location_id"
    },
    constant_field_values={
        "unit_name": "m^3/s",
        "variable_name": "streamflow_hourly_inst",
        "configuration_name": "usgs_observations"
    }
)

In [ ]:
# Load the crosswalk data
eval.load.import_location_crosswalks(
    in_path=CROSSWALK_FILEPATH
)

In [ ]:
# Load the secondary timeseries data and map over the fields and set constants
eval.load.import_secondary_timeseries(
    in_path=SECONDARY_TIMESERIES_FILEPATH,
    field_mapping={
        "reference_time": "reference_time",
        "value_time": "value_time",
        "configuration": "configuration_name",
        "measurement_unit": "unit_name",
        "variable_name": "variable_name",
        "value": "value",
        "location_id": "location_id"
    },
    constant_field_values={
        "unit_name": "m^3/s",
        "variable_name": "streamflow_hourly_inst",
        "configuration_name": "nwm30_retrospective"
    }
)

In [ ]:
# Add some attributes
eval.load.add_attribute(
    [
        Attribute(
            name="drainage_area",
            type="continuous",
            description="Drainage area in square kilometers"
        ),
        Attribute(
            name="ecoregion",
            type="categorical",
            description="Ecoregion"
        ),
        Attribute(
            name="year_2_discharge",
            type="continuous",
            description="2-yr discharge in cubic meters per second"
        ),
    ]
)

In [ ]:
# Load the location attribute data
eval.load.import_location_attributes(
    in_path=GEO_FILEPATH,
    field_mapping={"attribute_value": "value"},
    pattern="test_attr_*.parquet",
)

In [ ]:
# Create the joined timeseries
eval.create_joined_timeseries(execute_udf=True)

Timeseries, location crosswalks, and location attributes can also be loaded from netcdf files.

In [ ]:
# Create a new Evaluation object
shutil.rmtree(TEST_STUDY_DIR, ignore_errors=True)
TEST_STUDY_DIR.mkdir(parents=True, exist_ok=True)

ev = Evaluation(dir_path=TEST_STUDY_DIR)

# Enable logging
ev.enable_logging()

# Clone the template
ev.clone_template()

In [ ]:
# The primary locations file with geometry
# (TEEHR currently only supports formats readable by geopandas)
NETCDF_GEO_FILEPATH = Path(
    "/home/sam/git/teehr/tests/data/test_study/geo/summa_locations.parquet"
)
gpd.read_parquet(NETCDF_GEO_FILEPATH)

In [ ]:
# Load the locations and add the configuration and variable domain entries
ev.load.import_locations(in_path=NETCDF_GEO_FILEPATH)

ev.load.add_configuration(
    Configuration(
        name="summa",
        type="primary",
        description="Summa Runoff Data"
    )
)

ev.load.add_variable(
    Variable(
        name="runoff",
        long_name="runoff"
    )
)

In [ ]:
# Here is an example considered a primary timeseries file in netcdf format.
NETCDF_TIMESERIES_FILEPATH = Path(
    "/home/sam/git/teehr/tests/data/test_study/timeseries/summa.example.nc"
)
# Look at the contents of the netcdf file to determine the field mapping
xr.open_dataset(NETCDF_TIMESERIES_FILEPATH)

The TEEHR Timeseries table schema includes fields:

- reference_time
- value_time
- configuration_name
- unit_name
- variable_name
- value
- location_id

In [ ]:
# Map the fields in the netcdf file to the fields in the database, defining the constant values
summa_field_mapping = {
    "time": "value_time",
    "averageRoutedRunoff_mean": "value",
    "gru": "location_id"
}
summa_constant_field_values = {
    "unit_name": "m^3/s",
    "variable_name": "runoff",
    "configuration_name": "summa",
    "reference_time": None
}

In [ ]:
# Load the timeseries data, mapping over the fields and setting constants
ev.load.import_primary_timeseries(
    in_path=NETCDF_TIMESERIES_FILEPATH,
    field_mapping=summa_field_mapping,
    constant_field_values=summa_constant_field_values
)

In [ ]:
ev.primary_timeseries.to_pandas().head()